# Transportation

$$
\begin{align}
    \text{min} \quad & \sum_{i \in I}\sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    \quad & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \;i \in I, j \in J \\
\end{align}
$$

In [1]:
# Python native modules
import json

# Third-party packages
import pandas as pd  # Tabular data
import pyomo.environ as pyo

In [2]:
# Read input file and store in local variable `input_data`
with open("input_transp.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [3]:
# Create local variables to store input parameters
availabilities = input_data["availabilities"]
demands = input_data["demands"]

# Dictionary of costs indexed by tuples (origin, destination)
costs = {
    (c["from"], c["to"]): c["value"]
    for c in input_data["costs"]
}

In [4]:
availabilities["S2"] = 23

In [5]:
# Create model instance
model = pyo.ConcreteModel()

In [6]:
# Sets for suppliers I and customers J
model.I = pyo.Set(initialize=availabilities.keys())
model.J = pyo.Set(initialize=demands.keys())

In [7]:
# Paramters
model.b = pyo.Param(model.I, initialize=availabilities)
model.d = pyo.Param(model.J, initialize=demands)
model.c = pyo.Param(model.I, model.J, initialize=costs)

In [8]:
# Decision variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.J, within=pyo.NonNegativeReals)

In [9]:
# Supplier availablity constraints
def av_cstr(model, i):
    return sum(model.x[i, :]) <= model.b[i]


model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)

In [10]:
# Demand equality constraints
def dem_cstr(model, j):
    return sum(model.x[:, j]) + model.z[j] == model.d[j]


model.dem_cstr = pyo.Constraint(model.J, rule=dem_cstr)

In [11]:
# Objective functions
def art_obj(model):
    return sum(model.z[:])


model.art_obj = pyo.Objective(rule=art_obj, sense=pyo.minimize)


def obj(model):
    total_cost = sum(
        model.c[i, j] * model.x[i, j]
        for i in model.I
        for j in model.J
    )
    return total_cost


model.obj = pyo.Objective(rule=obj, sense=pyo.minimize)


model.obj.deactivate()
model.art_obj.activate()

In [12]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [13]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 2.0, 'Upper bound': 2.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
# Access the artificials optimized
K = model.art_obj()

In [15]:
def art_cstr(model):
    return sum(model.z[:]) <= K


model.art_cstr = pyo.Constraint(rule=art_cstr)

In [16]:
model.art_obj.deactivate()
model.obj.activate()

In [17]:
solver.solve(model)

{'Problem': [{'Lower bound': 501.0, 'Upper bound': 501.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [18]:
# Use objective as a callable to see its value
model.obj()

501.0

In [19]:
sol = [
    {"from": i, "to": j, "value": val}
    for (i, j), val in model.x.extract_values().items()
]

In [20]:
dataframe = pd.DataFrame(sol).pivot(
    index="from", columns="to", values="value"
)

In [21]:
display(dataframe)

to,C1,C2,C3,C4
from,,,,
S1,0.0,5.0,0.0,9.0
S2,0.0,8.0,15.0,0.0
S3,5.0,0.0,0.0,6.0
